In [1]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt

In [2]:
clust = pd.read_csv('evo/clusters.csv', header = None)

In [2]:
# with pd.HDFStore('evo/test1.h5',  mode='r') as newstore:
#     Y = newstore.select('Y')

In [3]:
cl = np.load('evo/cluster2go_data.npz')

In [4]:
cluster2go_data = csr_matrix((cl['data'], cl['indices'], cl['indptr']), shape=cl['shape'])

In [5]:
cluster2go_cols=pd.read_csv('evo/cluster2go_data.cols.txt',sep='\t', header=None, names=['feature'])
cluster2go_cols.head()

,feature
0,go_105
1,go_45454
2,go_6355
3,go_5975
4,go_6313


In [6]:
cluster2go_clusters = pd.read_csv('evo/cluster2go_data.rows.txt',sep='\t', header=None)

In [7]:
Y_dense = cluster2go_data.todense()

In [8]:
Y = pd.DataFrame(Y_dense,columns=cluster2go_cols['feature'].tolist())

In [9]:
Y.head()

,go_105,go_45454,go_6355,go_5975,go_6313,go_5992,go_8152,go_55114,go_9401,go_9404,...,go_48478,go_7096,go_30218,go_32007,go_30041,go_15780,go_9890,go_34775,go_71630,go_46649
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
Y["cluster"] = cluster2go_clusters

In [11]:
Y = Y[Y['cluster'].isin(clust.loc[:,0].tolist())]

In [12]:
Y.shape

(19165021, 1455)

In [ ]:
#relationships

In [13]:
relationship_df = pd.read_csv('evo/go_relations.txt', sep='\t')

In [14]:
relationship_df.head()

,TERM,PARENT_TERM
0,go_60974,go_60973
1,go_61345,go_60071
2,go_61345,go_61341
3,go_43554,go_9060
4,go_43554,go_15975


In [15]:
parents = set()
childrens = set()
for _, parent_child_row in relationship_df.iterrows():
    parents.add(parent_child_row['PARENT_TERM'])
    childrens.add(parent_child_row['TERM'])


In [16]:
only_childrens = childrens - parents

In [17]:
len(only_childrens)

13748

In [18]:
list(only_childrens)[0:10]

['go_33466',
 'go_90265',
 'go_35423',
 'go_10493',
 'go_19462',
 'go_43316',
 'go_18233',
 'go_46773',
 'go_55089',
 'go_10978']

In [19]:
all_columns = cluster2go_cols['feature'].tolist()
no_children_columns =  [col for col in  all_columns if col not in only_childrens]

In [20]:
len(no_children_columns)

1029

In [21]:
Y.drop(no_children_columns, inplace=True,axis=1)

In [22]:
Y.shape

(19165021, 426)

In [24]:
Y.head()

,go_105,go_45454,go_9401,go_45227,go_42128,go_6189,go_19354,go_71722,go_9231,go_19544,...,go_30150,go_244,go_45122,go_48484,go_71929,go_48478,go_32007,go_30041,go_71630,cluster
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,cl_357047299
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,cl_357037677
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,cl_366996869
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,cl_357040854
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,cl_357034477


In [25]:
with pd.HDFStore('evo/Y.h5',  mode='w') as store:
    store.put('Y', Y)

In [26]:
X = pd.read_csv('evo/calculated_features.txt.gz',sep='\t')

In [27]:
X.head()
#Y = Y[Y['cluster'].isin(clust.loc[:,0].tolist())]

,cluster_is,A,Acidic,Aliphatic,Aromatic,Basic,C,Charged,D,E,...,pseb_H2_l_14,pseb_H2_l_15,pseb_H2_l_16,pseb_H2_l_17,pseb_H2_l_18,pseb_H2_l_19,pseb_H2_l_20,relative_codone_usage,repeated,tmlen
0,cl_375983605,0.05,3.571,25.893,6.250,18.750,0.04,22.321,0.02,0.02,...,-8.513,20.836,11.376,-15.557,0.234,26.643,9.762,0.464467,1,0
1,cl_375983606,0.07,6.780,30.508,10.169,18.644,0.02,25.424,0.05,0.02,...,-19.173,14.249,9.855,-18.432,-9.204,0.403,13.919,0.360894,1,0
2,cl_375983604,0.17,5.310,44.248,10.619,8.850,0.02,14.159,0.04,0.01,...,2.466,-0.382,-5.318,7.595,1.659,11.356,1.623,0.335273,1,24
3,cl_375983602,0.06,1.961,11.765,11.765,1.961,0.00,3.922,0.02,0.00,...,-1.114,2.396,2.498,4.174,-4.670,2.988,2.033,0.197222,0,0
4,cl_375983358,0.06,16.418,27.761,13.433,13.731,0.00,30.149,0.07,0.09,...,6.242,-2.736,4.083,-3.102,-1.884,2.836,6.543,0.701023,2,0


In [28]:
X = X[X['cluster_is'].isin(clust.loc[:,0].tolist())]

In [29]:
X.shape, Y.shape

((19165021, 62), (19165021, 426))

In [ ]:
with pd.HDFStore('evo/X.h5',  mode='w') as store:
    store.put('X', X)